In [4]:
# 对lightGBM使用交叉验证和GridSearchCV调参；
# 调参前：0.1234，调参后：0.1220
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

df5 = pd.read_csv('train_TTI_6+6+3_more.csv')
df5['time']=pd.to_datetime(df5['time'])
df5['month']=[x.month for x in df5['time']]

In [3]:
# 利用GridSearchCV()搜索较好的参数。
train_train=df5
train_train['hour']=[x.hour+x.minute/60 for x in train_train['time']] # 需要time列转化为datetime类
#train_test['hour']=[x.hour+x.minute/60 for x in train_test['time']] # 需要time列转化为datetime类
mae=0
for id_road,group in train_train.groupby('id_road'):
    #X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group[X_columns_list].values
    y=group[y_columns_list].values
    y0=y[:,0]
    y1=y[:,1]
    y2=y[:,2]
    model0= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model1= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model2= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    params_test0={
                #'min_child_samples': [18,19,20,21,22]
                #'num_leaves': [31,40，5063],
                #'max_depth': [4,7，10]
              #'learning_rate': [0.01, 0.02,0.08,0.15],
              #'n_estimators':[100,500,1000]
    }
    gsearch0=GridSearchCV(estimator=model0, param_grid=params_test0 ,scoring='neg_mean_absolute_error', cv=6)#, verbose=1, n_jobs=4)
    gsearch0.fit(X,y0)
    print(gsearch0.best_score_)
    print(gsearch0.best_params_)

-0.05495542791258303
{'max_depth': 7}
-0.060218029322634564
{'max_depth': 7}
-0.052792845062619026
{'max_depth': 7}
-0.06568675308235891
{'max_depth': 7}
-0.04317767192696955
{'max_depth': 7}
-0.046168985124148264
{'max_depth': 7}
-0.06246818120476807
{'max_depth': 7}
-0.12176774426879505
{'max_depth': 4}
-0.03946882163968926
{'max_depth': 4}
-0.061652130810510734
{'max_depth': 7}
-0.1478458985460462
{'max_depth': 4}
-0.3240937239241952
{'max_depth': 7}


In [38]:


month_list=[1,2,3,10,11,12]
mae_sum=0
for i in month_list:
    train_train=df5.loc[df5['month']!=i]
    train_train.drop(['month'],axis=1,inplace=True)
    train_test=df5.loc[df5['month']==i]
    train_test.drop(['month'],axis=1,inplace=True)

    train_train['hour']=[x.hour+x.minute/60 for x in train_train['time']] # 需要time列转化为datetime类
    train_test['hour']=[x.hour+x.minute/60 for x in train_test['time']] # 需要time列转化为datetime类

    train_train=train_train.loc[train_train['hour']>=7.5]
    train_train=train_train.loc[train_train['hour']<=22]
    train_test=train_test.loc[train_test['hour']>=7.5]
    train_test=train_test.loc[train_test['hour']<=22]
    mae=0
    for id_road,group in train_train.groupby('id_road'):
        #X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        y_columns_list=['TTI6','TTI7','TTI8']
        X=group[X_columns_list].values
        y=group[y_columns_list].values
        y0=y[:,0]
        y1=y[:,1]
        y2=y[:,2]
        model0= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        model1= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        model2= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        #model = RandomForestRegressor(max_features="auto",n_estimators=50, min_samples_leaf = 10)
        #model=DecisionTreeRegressor(criterion='mae',splitter='random',max_depth=10,min_samples_split=5,min_samples_leaf=4)
        #model = linear_model.LinearRegression()
        model0.fit(X, y0)
        model1.fit(X, y1)
        model2.fit(X, y2)
        date_test=train_test.loc[train_test['id_road']==id_road]
        X_test=date_test[X_columns_list].values
        y_test=date_test[y_columns_list].values
        y0_predict=model0.predict(X_test)
        y1_predict=model1.predict(X_test)
        y2_predict=model2.predict(X_test)
        #y_predict=model.predict(X_test)
        #err=np.abs(y_test-y_predict)
        #mae+=np.sum(np.sum(err))
        err0=np.abs(y_test[:,0]-y0_predict)
        err1=np.abs(y_test[:,1]-y1_predict)
        err2=np.abs(y_test[:,2]-y2_predict)
        mae0=np.sum(err0)
        mae1=np.sum(err1)
        mae2=np.sum(err2)
        #print(mae0/len(err0),mae1/len(err1),mae2/len(err2))
        mae+=mae0+mae1+mae2
    mae/=len(train_test)*3
    print(mae)
    mae_sum+=mae
print(mae_sum/len(month_list))

0.19388257237873838
0.104527610957374
0.13459852580973544
0.13056445859501145
0.134598146756048
0.13725844614118718
0.1392382934396824


In [28]:
# 验证递推3个y是否会有提升。
# 结论：无提升。

#month_list=[1,2,3,10,11,12]
month_list=[1]
mae_sum=0
for i in month_list:
    train_train=df5.loc[df5['month']!=i]
    train_train.drop(['month'],axis=1,inplace=True)
    train_test=df5.loc[df5['month']==i]
    train_test.drop(['month'],axis=1,inplace=True)
    train_train['hour']=[x.hour+x.minute/60 for x in train_train['time']] # 需要time列转化为datetime类
    train_test['hour']=[x.hour+x.minute/60 for x in train_test['time']] # 需要time列转化为datetime类
    mae=0
    for id_road,group in train_train.groupby('id_road'):
        #X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        X1_columns_list=['TTI1','TTI2','TTI3','TTI4','TTI5','TTI6','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        X2_columns_list=['TTI2','TTI3','TTI4','TTI5','TTI6','TTI7','speed0','speed1','speed2','speed3','speed4','speed5','hour','isHoliday','weekday']
        y_columns_list=['TTI6','TTI7','TTI8']
        X=group[X_columns_list].values
        X1=group[X1_columns_list].values
        X2=group[X2_columns_list].values
        y=group[y_columns_list].values
        y0=y[:,0]
        y1=y[:,1]
        y2=y[:,2]
        model0= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        model1= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        model2= LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
        #model = RandomForestRegressor(max_features="auto",n_estimators=50, min_samples_leaf = 10)
        #model=DecisionTreeRegressor(criterion='mae',splitter='random',max_depth=10,min_samples_split=5,min_samples_leaf=4)
        #model = linear_model.LinearRegression()
        model0.fit(X, y0)
        model1.fit(X1, y1)
        model2.fit(X2, y2)
        date_test=train_test.loc[train_test['id_road']==id_road]
        y_test=date_test[y_columns_list].values # 先取出3个y

        X_test=date_test[X_columns_list].values
        y0_predict=model0.predict(X_test)
        date_test['TTI6']=y0_predict
        X1_test=date_test[X1_columns_list].values
        y1_predict=model1.predict(X1_test)
        date_test['TTI7']=y1_predict
        X2_test=date_test[X2_columns_list].values
        y2_predict=model2.predict(X2_test)
        #y_predict=model.predict(X_test)
        #err=np.abs(y_test-y_predict)
        #mae+=np.sum(np.sum(err))
        err0=np.abs(y_test[:,0]-y0_predict)
        err1=np.abs(y_test[:,1]-y1_predict)
        err2=np.abs(y_test[:,2]-y2_predict)
        mae0=np.sum(err0)
        mae1=np.sum(err1)
        mae2=np.sum(err2)
        print(mae0/len(err0),mae1/len(err1),mae2/len(err2))
        mae+=mae0+mae1+mae2
    mae/=len(train_test)*3
    print(mae)
    mae_sum+=mae
print(mae_sum/len(month_list))

0.08910520428021439 0.12656957639385394 0.14481976001792557
0.07984482707392836 0.11064597008705178 0.1284649191139318
0.055838958768116925 0.08707099884222948 0.10634023519100656
0.09023131271227146 0.12936658871186116 0.15532192223040095
0.04977573191528718 0.06334475177210502 0.06927189942448385
0.07327343917362776 0.09827919875619018 0.11193523239895035
0.20174421104584817 0.2309332726941033 0.2450869964138369
0.13322241849441632 0.17792269546980624 0.19582591527484589
0.06427611962391266 0.07729850769472102 0.08266087396115586
0.08670420687197367 0.11095820052787285 0.12588681436193577
0.19116705421528513 0.23241630984320627 0.2481824747191277
0.5077567762032622 0.660208707967827 0.7343277279202115
0.16877999472685512
0.16877999472685512


In [5]:
# 训练代码
#data_train=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','isHoliday','weekday']
#Xy_test=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','id_sample0','id_sample1','id_sample2','isHoliday','weekday']
data_train = df5
data_train['hour']=[x.hour+x.minute/60 for x in data_train['time']]    # 只保留小时和分
#display(data_train)
Xy_test=pd.read_csv('test_6+6+3_more.csv')
Xy_test['time']=pd.to_datetime(Xy_test['time'])
Xy_test['hour']=[x.hour+x.minute/60 for x in Xy_test['time']]    # 只保留小时和分
y_predict=np.zeros(Xy_test.shape[0]*3)
for id_road,group in Xy_test.groupby('id_road'):
    group_train=data_train.loc[data_train['id_road']==id_road]
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','hour']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group_train[X_columns_list].values
    y=group_train[y_columns_list].values
    y0=y[:,0]
    y1=y[:,1]
    y2=y[:,2]
    model0= LGBMRegressor(num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model1= LGBMRegressor(num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model2= LGBMRegressor(num_leaves=31,max_depth=7,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model0.fit(X, y0)
    model1.fit(X, y1)
    model2.fit(X, y2)
    id_sample_columns_list=['id_sample0','id_sample1','id_sample2']
    X_test=group[X_columns_list].values
    id_sample=group[id_sample_columns_list].values
    y0_predict=model0.predict(X_test)
    y1_predict=model1.predict(X_test)
    y2_predict=model2.predict(X_test)
    
    for i in range(np.size(y0_predict)):
        y_predict[int(id_sample[i,0])]=y0_predict[i]
        y_predict[int(id_sample[i,1])]=y1_predict[i]
        y_predict[int(id_sample[i,2])]=y2_predict[i]
y_predict

array([1.07123789, 1.08742199, 1.10304319, ..., 1.60532471, 1.62759496,
       1.55680951])

In [35]:
# 输出y_predict
out=np.zeros([len(y_predict),2])
for i in range(len(y_predict)):
    out[i,0]=i
    out[i,1]=y_predict[i]
out=pd.DataFrame(out,columns=['id_sample','TTI'])
out.to_csv('submit19.csv',mode='w',index=False,header=True,encoding= 'utf-8')